In [96]:
import pandas as pd 
import os
import psycopg2
import numpy as np

pd.options.mode.chained_assignment = None 

In [97]:
# utility methods
def get_db_list(cur):
    cur.execute("SELECT datname from pg_database;")
    return cur.fetchall()


def con_check(cur, my_db):
    cur.execute("SELECT * FROM pg_stat_activity WHERE datname = '{}';".format(my_db))
    return cur.fetchall()


def show_current_db(cur):
    cur.execute("SELECT current_database()")
    return cur.fetchall()


def schema_list(cur, schema):
    cur.execute('''
          SELECT EXISTS(SELECT 1 FROM information_schema.schemata 
            WHERE schema_name = '{}');
            '''.format(schema))
    return cur.fetchall()

def show(df):
    print(df.head(), df.shape)

prompt_create = '''CREATE TABLE IF NOT EXISTS {}.{} '''
prompt_delete = '''DROP TABLE IF EXISTS {}.{} ;'''
prompt_insert = '''INSERT INTO {}.{} '''


In [98]:
DB = 'student'
conn_string = "host=0.0.0.0 dbname={} user={} password={}"

conn = psycopg2.connect(conn_string.format(DB + 'db', DB, DB))
conn.set_session(autocommit=True)
cur = conn.cursor()

my_db = 'etldb'
print(con_check(cur, my_db))
cur.execute("DROP DATABASE IF EXISTS {}".format(my_db))
cur.execute("CREATE DATABASE {} WITH ENCODING 'utf8' TEMPLATE template0 CONNECTION LIMIT 3".format(my_db))
conn.close()

schema = 'work'
conn = psycopg2.connect(conn_string.format(my_db, DB, DB))
conn.set_session(autocommit=True)
cur = conn.cursor()
cur.execute("CREATE SCHEMA IF NOT EXISTS {};".format(schema))
cur.execute("SET search_path = work, public;")
print("DB created successfully")

[]
DB created successfully


In [99]:
# File locations to read data 
data_dir = os.path.join(os.getcwd(), 'data')
sas_path = os.path.join(data_dir, 'sas_data')
airport_mismatch_path = os.path.join(data_dir, r'mismatch.csv')
country_detail_path = os.path.join(data_dir, r'country_details.csv')
airport_data = os.path.join(data_dir, 'airport-codes_csv.csv')
arrival_port_map = os.path.join(data_dir, 'arrival_port_map.csv')
demographic_data = os.path.join(data_dir, 'us-cities-demographics.csv')
sas_data = [os.path.join(sas_path, x) for x in os.listdir(sas_path) if x.startswith('part')]

In [100]:
table_name = "fact_table"
sql = prompt_create.format(schema, table_name) + ''' (
            cicid int primary key,
            personId int,
            statusFlagId int,
            ident text,
            visaId int,
            arrivalDate timestamp,
            arrivalPortId int,
            countryId int,
            arrivalPort text,
            state_code text,
            departureDate timestamp,
            mode text,
            visatype text,
            gender text,
            state text,
            countryName text,
            airportName text
        )
        '''
cur.execute(sql)

In [101]:
table_name = "dim_person"
sql = prompt_create.format(schema, table_name) + ''' (
        personId text primary key,
        birthYear int,
        gender text
    )
'''
cur.execute(sql)

table_name = "dim_status"
sql = prompt_create.format(schema, table_name) + ''' (
    statusFlagId int primary key,     
    arrivalFlag text,
    departureFlag text,
    updateFlag text,
    matchFlag text
 )'''
cur.execute(sql)

table_name = "dim_country"
sql = prompt_create.format(schema, table_name) + '''(
    countryId int,
    countryName text
)'''

cur.execute(sql)

In [102]:
table_name = "dim_us_state"
sql = prompt_create.format(schema, table_name) + ''' (
        state_code text primary key,
        state text,
        median_age float,
        male_pop float,
        female_pop float, 
        total_pop float,
        vet_no float,
        foreign_born float,
        avg_house_size float
)'''
cur.execute(sql)

In [103]:
table_name = "dim_airport"
sql = prompt_create.format(schema, table_name) + ''' (
    ident text primary key,
    type  text,
    iata_code text,
    name text,
    iso_country text,
    iso_region text,
    municipality text,
    gps_code text,
    lat float,
    long float)
'''
cur.execute(sql)

In [104]:
table_name = "dim_arrival"
sql = prompt_create.format(schema, table_name) + ''' (
        arrivalPortId int primary key,
        arrival_port text,
        num_arrivals int,
        state_code text
        )'''
cur.execute(sql)

table_name = "dim_time"
sql = prompt_create.format(schema, table_name) + ''' (
    arrivalDate timestamp primary key,
    year int,
    month int,
    day int,
    week int,
    weekday text)'''
cur.execute(sql)

In [105]:
# creating the demographic data
demographic_df = pd.read_csv(demographic_data, sep=';')
demographic_df.head(3)

insert_into_us_state = '''
INSERT INTO {}.{}
({})                   
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
'''

dem = demographic_df.drop(['Race', 'Count', 'City'], axis=1).drop_duplicates()
renaming_dict = {
    'State': 'state',
    'Median Age': 'median_age',
    'Male Population': 'male_pop',
    'Female Population': 'female_pop',
    'Total Population': 'total_pop',
    'Number of Veterans': 'vet_no',
    'Foreign-born': 'foreign_born',
    'Average Household Size': 'avg_house_size',
    'State Code': 'state_code'
}
# inserting the data in dim_us_state

dem = dem.rename(columns=renaming_dict)
cols = dem.columns.difference(['state', 'state_code'])
funcs = {x: np.sum for x in cols}
funcs['avg_house_size'], funcs['median_age'] = np.mean, np.median
dem = dem.groupby(['state', 'state_code'], as_index=False).agg(funcs)


cols = dem.columns.tolist()
insert_statement = insert_into_us_state.format(schema, 'dim_us_state', ','.join(cols))

for indx, row in dem.iterrows():
    cur.execute(insert_statement, row)
print("Successfully inserted data in dim_us_state table")

Successfully inserted data in dim_us_state table


In [106]:
dem.head(1).T

,0
state,Alabama
state_code,AL
avg_house_size,2.434286
female_pop,552381.0
foreign_born,52154.0
male_pop,497248.0
median_age,38.0
total_pop,1049629
vet_no,71543.0


In [107]:
immigration_df = pd.read_parquet(sas_data[0])
cols = sorted(immigration_df.columns)
print(immigration_df.shape)

(220160, 28)


In [108]:
# shaping the immigration data for further use
renaming_dict = {
    "i94yr": "year",
    "i94mon": "month",
    "i94cit": "bornCountry",
    "i94res": "residentCountry",
    "i94port": "arrivalPort",
    "i94mode": "mode",
    "i94addr": "arrivalAddress",
    "i94bir": "age",
    "i94visa": "visaId",
    "entdepa": "arrivalFlag",
    "entdepd": "departureFlag",
    "entdepu": "updateFlag",
    "matflag": "matchFlag",
    "biryear": "birthYear",
    "fltno": "flightNumber", 
    "visatype": "visaType"
}
# rename
imm_df = immigration_df.rename(columns=renaming_dict).iloc[:300]

#cast
casting_cols = ["cicid","year", "month", "bornCountry", "residentCountry", "mode", 
                "age","visaId","birthYear", "arrdate", "depdate"]
imm_df = imm_df.dropna(subset=casting_cols)
imm_df[casting_cols] = imm_df[casting_cols].astype(int)
date_mapper = lambda x: pd.DateOffset(days=x) + pd.Timestamp('1960-01-01')
imm_df["arrivalDate"] = imm_df['arrdate'].map(date_mapper)
imm_df["departureDate"] = imm_df["depdate"].map(date_mapper)
imm_df["flight_no"] = imm_df.apply(lambda row: row['airline'] + str(int(row['flightNumber'])), axis=1)

#drop
drop_cols = ["arrdate", "depdate", "count", "admnum", "dtadfile", "visapost", "occup", "dtaddto", "insnum"]
imm_df = imm_df.drop(drop_cols, axis=1)

In [109]:
im = imm_df[['flight_no','arrivalPort']].drop_duplicates().sort_values(by=['flight_no', 'arrivalPort'])

In [110]:
# creating the airport table
airport_df = pd.read_csv(airport_data)
m1 = airport_df['iso_country'] == 'US'
m2 = ~pd.isnull(airport_df['iata_code'])
m3 = airport_df["type"].isin(["large_airport", "medium_airport", "small_airport"])
air_df = airport_df.loc[(m1 & m2 & m3)]

air_df['iso_region'] = air_df['iso_region'].str[3:] 
air_df['lat'] = air_df['coordinates'].str.split(',').str[0]
air_df['long'] = air_df['coordinates'].str.split(',').str[1]

air_df[['lat', 'long']] = air_df[['lat', 'long']].astype(float)


drop_columns = ['local_code', 'elevation_ft', 'continent', 'coordinates']
air_df = air_df.drop(drop_columns, axis=1).dropna().drop_duplicates()

# inserting data in airport table
cols = air_df.columns.tolist()

insert_into_airport_table = '''
INSERT INTO {}.{}
({})                   
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
'''.format(schema, 'dim_airport', ','.join(cols))

for indx, row in air_df.iterrows():
    cur.execute(insert_into_airport_table, row)

print("Successfully inserted data into airport table")

air_df[air_df['iata_code'] == 'MSP']

Successfully inserted data into airport table


,ident,type,name,iso_country,iso_region,municipality,gps_code,iata_code,lat,long
28131,KMSP,large_airport,Minneapolis-St Paul International/Wold-Chamber...,US,MN,Minneapolis,KMSP,MSP,-93.221802,44.882


In [111]:
air_df.head(1).T

,440
ident,07FA
type,small_airport
name,Ocean Reef Club Airport
iso_country,US
iso_region,FL
municipality,Key Largo
gps_code,07FA
iata_code,OCA
lat,-80.274803
long,25.325399


In [112]:
# Join airport table to immigration table
air_df_to_map = air_df.loc[:, ['ident', 'iata_code', 'name']].rename(columns={'name': 'airportName'})
mask = im['arrivalPort'].isin(air_df_to_map['iata_code'].unique())
im2_merged = im[mask]
im2_unmerged = im[~mask]
airport_mismatch_df = pd.read_csv(airport_mismatch_path, index_col=0).squeeze("columns")
im2_merged = im2_merged.merge(air_df_to_map, left_on='arrivalPort', right_on='iata_code')
im2_unmerged.insert(loc=2, column='arrivalPort2', value=im2_unmerged['flight_no'].map(lambda c: airport_mismatch_df[c]).tolist())
im2_unmerged = im2_unmerged.merge(air_df_to_map, left_on='arrivalPort2', right_on='iata_code').drop(['arrivalPort2'], axis=1)
im2 = pd.concat([im2_merged, im2_unmerged], axis=0).dropna()
show(im2)


  flight_no arrivalPort ident iata_code                  airportName
0    AA1250         MIA  KMIA       MIA  Miami International Airport
1    AA1322         MIA  KMIA       MIA  Miami International Airport
2    AA1481         MIA  KMIA       MIA  Miami International Airport
3    AA1502         MIA  KMIA       MIA  Miami International Airport
4    AA1558         MIA  KMIA       MIA  Miami International Airport (88, 5)


In [113]:
# clean data
m1 = imm_df['age'] >= 0
m2 = ~imm_df['cicid'].isna()
m3 = ~(imm_df['gender'].isna() | imm_df['birthYear'].isna())
imm_df = imm_df.loc[m1 & m2 & m3].drop_duplicates(subset=['cicid'])
print("Data cleaning complete, shape {}".format(imm_df.shape))

Data cleaning complete, shape (185, 22)


In [114]:
# creating the time table
time_table = imm_df.loc[:, ['arrivalDate']]
def transform(row):
    t = pd.Timestamp(row['arrivalDate'], unit='ms')
    row['arrivalDate'] = t
    row['day'] = t.day
    row['month'] = t.month
    row['year'] = t.year
    row['week'] = t.weekofyear
    row['weekday'] = t.day_name()
    return row
time_table = time_table.apply(transform, axis=1)


# insert the data in time table
cols = time_table.columns.tolist()
insert_into_time_table = '''
INSERT INTO {}.{}
({})                   
VALUES (%s, %s, %s, %s, %s, %s) ON CONFLICT(arrivalDate) DO NOTHING
'''.format(schema, 'dim_time', ','.join(cols))

for indx, row in time_table.iterrows():
    cur.execute(insert_into_time_table, list(row))
    
print("Successfully inserted data into time table")

Successfully inserted data into time table


In [115]:
time_table.head(1).T

,4
arrivalDate,2016-04-08 00:00:00
day,8
month,4
year,2016
week,14
weekday,Friday


In [116]:
# creating the person table
person_table = imm_df.loc[:, ['birthYear','gender']].dropna().drop_duplicates()
person_table['personId'] = np.arange(person_table.shape[0])
show(person_table)
cols = person_table.columns.tolist()

# insert the data in person table
insert_into_person_table = '''
INSERT INTO {}.{}
({})                   
VALUES (%s, %s, %s)
'''.format(schema, 'dim_person', ','.join(cols))

for indx, row in person_table.iterrows():
    cur.execute(insert_into_person_table, row)

print("Successfully inserted data into person table")

    birthYear gender  personId
4        1974      M         0
42       1970      F         1
57       1980      F         2
65       1984      M         3
79       1966      F         4 (92, 3)
Successfully inserted data into person table


In [117]:
person_table.head(1).T

,4
birthYear,1974
gender,M
personId,0


In [118]:
# creating the country table
country_detail = pd.read_csv(country_detail_path,  header = None, index_col = 0).squeeze("columns")
country_table = imm_df.loc[:, ['bornCountry']].drop_duplicates().rename(columns={'bornCountry': 'countryId'})
country_table['countryName'] = country_table['countryId'].map(lambda x: country_detail[x])
show(country_table)

# insert the data in country table
cols=country_table.columns
insert_into_country_table = '''
INSERT INTO {}.{}
({})                   
VALUES (%s, %s)
'''.format(schema, 'dim_country', ','.join(cols))

for indx, row in country_table.iterrows():
    cur.execute(insert_into_country_table, row)

print("Successfully inserted data into country table")

   countryId countryName
4        117       ITALY (1, 2)
Successfully inserted data into country table


In [119]:
country_table.head(1).T


,4
countryId,117
countryName,ITALY


In [120]:
# creating the arrival table
arrival_data_map = pd.read_csv(arrival_port_map, header = None, index_col = 0).squeeze("columns")
arrival_port_table = imm_df.groupby('arrivalPort', as_index=False)['arrivalAddress'].count().dropna().drop_duplicates()

arrival_port_table['arrivalPortId'] = np.arange(arrival_port_table.shape[0])
arrival_port_table['stateCode'] = arrival_port_table['arrivalPort'].map(lambda x: arrival_data_map[x])

arrival_port_table = arrival_port_table.rename(columns={"arrivalAddress": "num_arrivals", 
                                                        "arrivalPort": "arrival_port",
                                                       "stateCode": "state_code"})

show(arrival_port_table)

# insert the data in arrival table
cols = arrival_port_table.columns
insert_into_arrival_table = '''
INSERT INTO {}.{}
({})                   
VALUES (%s, %s, %s, %s)
'''.format(schema, 'dim_arrival', ','.join(cols))

for indx, row in arrival_port_table.iterrows():
    cur.execute(insert_into_arrival_table, row)

print("Successfully inserted data into arrival table")



  arrival_port  num_arrivals  arrivalPortId state_code
0          HOU             1              0         TX
1          LOS            55              1         CA
2          LVG            10              2         NV
3          MIA            72              3         FL
4          NYC            23              4         NY (8, 4)
Successfully inserted data into arrival table


In [121]:
arrival_port_table.head(1).T

,0
arrival_port,HOU
num_arrivals,1
arrivalPortId,0
state_code,TX


In [122]:
# creating the status table
status_table = imm_df.loc[:, ['arrivalFlag','departureFlag', 'updateFlag','matchFlag']]
status_table = status_table.drop_duplicates()
status_table['statusFlagId'] = np.arange(status_table.shape[0])
show(status_table)

# insert data into status table

cols = status_table.columns
insert_into_arrival_table = '''
INSERT INTO {}.{}
({})                   
VALUES (%s, %s, %s, %s, %s)
'''.format(schema, 'dim_status', ','.join(cols))

for indx, row in status_table.iterrows():
    cur.execute(insert_into_arrival_table, row)

print("Successfully inserted data into status table")


   arrivalFlag departureFlag updateFlag matchFlag  statusFlagId
4            O             O       None         M             0
79           T             I       None         M             1
80           T             K       None         M             2
82           T             O       None         M             3
84           G             O       None         M             4 (9, 5)
Successfully inserted data into status table


In [123]:
status_table.head(1).T

,4
arrivalFlag,O
departureFlag,O
updateFlag,None
matchFlag,M
statusFlagId,0


In [124]:
# creating the fact table
cols = ['cicid', 'arrivalDate','departureDate','mode','bornCountry', 'airline','birthYear',
        'flightNumber','visaId','visaType','gender','arrivalPort','arrivalFlag','departureFlag', 
        'updateFlag', 'matchFlag', 'flight_no']

# adding time
small_table = imm_df.loc[:, cols].drop_duplicates(subset=['cicid'])
print('Added time', small_table.shape)
small_table[['gender', 'birthYear']]

Added time (185, 17)


,gender,birthYear
4,M,1974
42,F,1970
57,F,1980
65,M,1984
79,F,1966
...,...,...
295,M,1985
296,F,1986
297,F,1988
298,F,1991


In [125]:

# adding person id 
cols = ['gender', 'birthYear']
small_table = small_table.merge(person_table , left_on=cols, right_on=cols)
print('Added person', small_table.shape)


Added person (185, 18)


In [126]:

# adding status flag id
cols = ['arrivalFlag','departureFlag','updateFlag','matchFlag']
small_table = small_table.merge(status_table, left_on=cols, right_on=cols)
print('Added status', small_table.shape)


Added status (185, 19)


In [127]:

# adding arrival port id
small_table = small_table.merge(arrival_port_table, left_on='arrivalPort',right_on='arrival_port')
print('Added arrival', small_table.shape)



Added arrival (185, 23)


In [128]:
# adding ident (airport)
im2_ = im2.loc[:, ['flight_no', 'ident', 'airportName']]
small_table = small_table.merge(im2_, left_on='flight_no', right_on='flight_no')
print('Added airport', small_table.shape)


Added airport (187, 25)


In [129]:

# add country
small_table = small_table.merge(country_table, left_on='bornCountry', right_on='countryId')
print('Added country', small_table.shape)



Added country (187, 27)


In [130]:
# add demographics
small_table = small_table.merge(dem.loc[:, ['state_code', 'state']], left_on='state_code', right_on='state_code')
print('Added demographics', small_table.shape)

print("Successfully joined dimension tables into fact table")

Added demographics (187, 28)
Successfully joined dimension tables into fact table


In [131]:
# Insert data into fact table
col_order = ['cicid', 'personId', 'statusFlagId', 'ident', 'visaId', 'arrivalDate', 'arrivalPortId', 'countryId','arrivalPort',
             'state_code','departureDate', 'mode', 'visaType', 'gender', 'state','countryName', 'airportName']
small_table = small_table.loc[:, col_order].drop_duplicates(subset=['cicid'])
small_table.head(1).T


,0
cicid,1360829
personId,0
statusFlagId,0
ident,KJFK
visaId,2
arrivalDate,2016-04-08 00:00:00
arrivalPortId,4
countryId,117
arrivalPort,NYC
state_code,NY


In [132]:
insert_into_fact_table = '''
INSERT INTO {}.{}
({})                   
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
'''.format(schema, 'fact_table', ','.join(col_order))

for indx, row in small_table.iterrows():
    cur.execute(insert_into_fact_table, row)
print("Successfully inserted data in fact table")

Successfully inserted data in fact table


In [133]:
show(small_table)

     cicid  personId  statusFlagId ident  visaId arrivalDate  arrivalPortId  \
0  1360829         0             0  KJFK       2  2016-04-08              4   
1  1360867         1             0  KJFK       2  2016-04-08              4   
2  1360882         2             0  KJFK       2  2016-04-08              4   
3  1360890         3             0  KJFK       2  2016-04-08              4   
4  1360921         0             4  KJFK       2  2016-04-08              4   

   countryId arrivalPort state_code departureDate  mode visaType gender  \
0        117         NYC         NY    2016-04-12     1       WT      M   
1        117         NYC         NY    2016-04-16     1       WT      F   
2        117         NYC         NY    2016-04-19     1       WT      F   
3        117         NYC         NY    2016-07-02     1       WT      M   
4        117         NYC         NY    2016-04-16     1       WT      M   

      state countryName                           airportName  
0  New Yor

In [140]:
# Quality check 
# Number of rows in the initial immigration dataframe and the total rows combined is dame
if small_table.shape[0] == imm_df.shape[0]:
    print('''
    Quality check passed, 
    Number of rows in immigration dataframe = {}
    Number of rows in fact table dataframe = {}'''.format(small_table.shape[0], imm_df.shape[0]))


    Quality check passed, 
    Number of rows in immigration dataframe = 185
    Number of rows in fact table dataframe = 185
